In [1]:
import scanpy as sc
import pandas as pd
import numpy as np
import cfp

/home/icb/dominik.klein/mambaforge/envs/cfp/lib/python3.11/site-packages/optuna/study/_optimize.py:29: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from optuna import progress_bar as pbar_module


In [2]:
adata=sc.read_h5ad("/lustre/groups/ml01/workspace/ot_perturbation/data/pbmc/pbmc_new_cytokine_processed.h5ad")

In [3]:
annotations = pd.read_csv("/lustre/groups/ml01/workspace/ten_million/data/data_2024_12_16/new_cell_type_annotations.csv", index_col=0)

/tmp/ipykernel_3711382/189709436.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  annotations = pd.read_csv("/lustre/groups/ml01/workspace/ten_million/data/data_2024_12_16/new_cell_type_annotations.csv", index_col=0)


In [4]:
annotations.head()

,sample,donor,treatment,cell_type,cell_type_new
89_001_008__s1,Donor1_4-1BBL,Donor1,cytokine,CD8 Naive,CD8 Naive
89_001_021__s1,Donor1_4-1BBL,Donor1,cytokine,CD4 Naive,CD4 Naive
89_001_061__s1,Donor1_4-1BBL,Donor1,cytokine,CD4 Naive,CD4 Naive
89_001_093__s1,Donor1_4-1BBL,Donor1,cytokine,Treg,Treg
89_001_112__s1,Donor1_4-1BBL,Donor1,cytokine,CD4 Memory,CD4 Memory


In [5]:
adata.obs["cell_type_new"] = annotations["cell_type_new"] 

In [6]:
rng = np.random.default_rng(0)
idcs_to_keep = []
for d in adata.obs["donor"].unique():
    adata_d = adata[adata.obs["donor"]==d]
    for ct in adata.obs["cell_type_new"].unique():
        adata_tmp = adata_d[adata_d.obs["cell_type_new"]==ct]
        idcs_to_keep.extend(list(rng.choice(adata_tmp.obs_names, min(1000, adata_tmp.n_obs), replace=False)))

In [7]:

import pickle

with open("/lustre/groups/ml01/workspace/ot_perturbation/data/pbmc/idcs_to_keep.pkl", "wb") as pickle_file:
    pickle.dump(idcs_to_keep, pickle_file)


In [8]:
with open("/lustre/groups/ml01/workspace/ot_perturbation/data/pbmc/idcs_to_keep.pkl", "rb") as pickle_file:
        idcs_to_keep2 = pickle.load(pickle_file)
    